# best

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob2 import glob  
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from pandasql import sqldf
import random
import os
from numba import jit

test_num = 24

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

&nbsp;

## 입력 shape 및 형태 정의 함수

In [2]:
@jit
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

@jit
def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [3]:
@jit
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [4]:
@jit
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0.2, mlp_dropout=0.2):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [5]:
@jit
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=30)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [6]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.001, verbose = 0):
    model = build_model(
    x_train.shape[1:],
    head_size=64,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[64],
    mlp_dropout=0.2,
    dropout=0.2,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU:1'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [7]:
@jit
def x_time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

@jit
def y_time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(14, len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values - df.iloc[index-1])

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [8]:
data_list = glob('./aT_data/data/train/*.csv')
epoch = 1000
batch = 64

In [9]:
def feature_select(df):
       #시간 코사인 인코딩
       if df['datadate'].all() == 0:
         df['datadate'] = 20180520
       df['datetime'] = pd.to_datetime(df['datadate'], format="%Y%m%d")
       df['cos_time'] = np.cos(2*np.pi*df['datetime'].dt.dayofyear/365)
       df = df.set_index('datetime')

       # 결측치
       df = df.replace(' ', np.nan)

       # 주산지 초기온도, 최대온도, 최저온도, 평균온도, 강수량, 습도
       df['주산지_초기온도'] = (df['주산지_0_초기온도(℃)'] + df['주산지_1_초기온도(℃)'] + df['주산지_2_초기온도(℃)'])/3
       df['주산지_최대온도'] = (df['주산지_0_최대온도(℃)'] + df['주산지_1_최대온도(℃)'] + df['주산지_2_최대온도(℃)'])/3
       df['주산지_최저온도'] = (df['주산지_0_최저온도(℃)'] + df['주산지_1_최저온도(℃)'] + df['주산지_2_최저온도(℃)'])/3
       df['주산지_평균온도'] = (df['주산지_0_평균온도(℃)'] + df['주산지_1_평균온도(℃)'] + df['주산지_2_평균온도(℃)'])/3
       df['주산지_강수량'] = (df['주산지_0_강수량(ml)'] + df['주산지_1_강수량(ml)'] + df['주산지_2_강수량(ml)'])/3

       # 필요한 컬럼 선택
       
       market_cols = ['단가(원)', '거래량'] # '경매건수',, '거래대금(원)'

       market_code_cols = [] # '도매시장코드', '도매법인코드', '산지코드 '

       market_statistics_cols = [ '일자별_도매가격_평균(원)', '해당일자_전체거래물량(kg)'
                                 ] 
       # '상위가격 평균가(원)',  '하위가격 평균가(원)',
       # '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)', '일자별_소매가격_최소( 원)',
       #  '하위가격 거래물량(kg)', '상위가격 거래물량(kg)',
       #  '일자별_도매가격_최대(원)','일자별_도매가격_최소(원)', , 
       
       export_cols = [] # '수출중량(kg)', '수출금액(달러)', '수입중량(kg)', '수입금액(달러)', '무역수지(달러)'
       
       date_cols = ['cos_time'] 
       # '일자구분_중순', '일자구분_초순', '일자구분_하순', '월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월','월구분_3월', '월구분_4월', '월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월', 'datetime'

       weather_cols = ['주산지_평균온도', '주산지_강수량'] 
       # '주산지_초기온도', '주산지_최대온도', '주산지_최저온도', 
      #  '주산지_0_초기온도(℃)', '주산지_1_초기온도(℃)', '주산지_2_초기온도(℃)',
      #  '주산지_0_최대온도(℃)', '주산지_1_최대온도(℃)', '주산지_2_최대온도(℃)',
      #  '주산지_0_최저온도(℃)', '주산지_1_최저온도(℃)', '주산지_2_최저온도(℃)',
      #  '주산지_0_평균온도(℃)', '주산지_1_평균온도(℃)', '주산지_2_평균온도(℃)',
      #  '주산지_0_강수량(ml)', '주산지_1_강수량(ml)', '주산지_2_강수량(ml)',
      #  '주산지_0_습도(%)', '주산지_1_습도(%)', '주산지_2_습도(%)'

       selected_columns = ['해당일자_전체평균가격(원)']  + market_cols + market_code_cols + market_statistics_cols + export_cols +  date_cols + weather_cols # train 에서 사용하지 않는 열
       df = df[[c for c in df.columns if c in selected_columns]]

       return df

In [10]:
def feature_na(df):
    df = df.interpolate()
    df = df.fillna(method = 'bfill')

    return df

&nbsp;

## Train 과정

In [11]:
losses = []
scalers = {}

for i in tqdm(data_list):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)
    df = feature_select(df)
    df = feature_na(df)

    # 변수와 타겟 분리
    X, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']
    
    scaler = RobustScaler()
    X = pd.DataFrame(scaler.fit_transform(X))
    scalers[df_number] = scaler

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    # y = y[14:]

    # time series window 생성
    data_x = x_time_window(X, 13, 1)
    data_y = y_time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y
    
    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=True, random_state=119)
 
    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch, learning_rate = 0.001, verbose = 1)
    
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')
    loss = transformer.evaluate(astype_data(x_val), y_val, verbose=0)
    print('\nvalid_loss: ', loss)
    losses.append(loss)

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

  0%|          | 0/37 [00:00<?, ?it/s]

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-09-28 22:13:19.584580: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-28 22:13:19.584682: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/1000


2022-09-28 22:13:20.351291: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-28 22:13:20.496247: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

17/17 [==============================] - 1s 25ms/step - loss: 25911.8027 - val_loss: 21076.0156
Epoch 2/1000
10/17 [===============>..............] - ETA: 0s - loss: 25951.8750

2022-09-28 22:50:17.603417: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 0s 10ms/step - loss: 25903.5957 - val_loss: 21064.7969
Epoch 3/1000
17/17 [==============================] - 0s 11ms/step - loss: 25894.2031 - val_loss: 21051.0684
Epoch 4/1000
17/17 [==============================] - 0s 10ms/step - loss: 25882.1660 - val_loss: 21033.0410
Epoch 5/1000
17/17 [==============================] - 0s 10ms/step - loss: 25866.1660 - val_loss: 21009.6230
Epoch 6/1000
17/17 [==============================] - 0s 10ms/step - loss: 25845.0801 - val_loss: 20979.5059
Epoch 7/1000
17/17 [==============================] - 0s 11ms/step - loss: 25817.8184 - val_loss: 20941.7109
Epoch 8/1000
17/17 [==============================] - 0s 9ms/step - loss: 25783.8789 - val_loss: 20896.1055
Epoch 9/1000
17/17 [==============================] - 0s 9ms/step - loss: 25743.8105 - val_loss: 20843.6152
Epoch 10/1000
17/17 [==============================] - 0s 9ms/step - loss: 25698.3770 - val_loss: 20785.4922
Epoch 11/1000
17/17 [=============

100%|██████████| 37/37 [37:20<00:00, 60.55s/it]


valid_loss:  19050.4453125


In [12]:
losses

[144254.96875,
 161745.296875,
 41828.96484375,
 57312.32421875,
 5394135.0,
 708823.5625,
 2946867.25,
 1918623.125,
 429348.75,
 44537152.0,
 87792.8046875,
 204568.328125,
 309011.625,
 227827.703125,
 117023.1796875,
 58669.0546875,
 794103.875,
 2634555.25,
 1952061.875,
 229107.625,
 57602.703125,
 459040.84375,
 4089997.5,
 195589.0625,
 36769.85546875,
 1182011.875,
 219508.53125,
 1225104.0,
 3709058.0,
 479539.8125,
 307521.75,
 1408074.625,
 1457877.625,
 145304.59375,
 13783.1103515625,
 36915496.0,
 19050.4453125]

&nbsp;

## Test 과정

In [13]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./aT_data/data/test/set_{i}/*.csv')
    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)
        
        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)

        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택 및 index 설정

        df = feature_select(df)
        df = feature_na(df)

        df = df[[c for c in df.columns if c != '해당일자_전체평균가격(원)']]
        
        df = pd.DataFrame(scalers[file_number].transform(df))

        # x_test  생성
        df = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))

        # model test
        if os.path.exists(f'./model_output/{test_num}') == False:
            os.mkdir(f'./model_output/{test_num}')

        if os.path.exists(f'./model_output/{test_num}/set_{i}') == False:
            os.mkdir(f'./model_output/{test_num}/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df)

        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/{test_num}/set_{i}/predict_{file_number}.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 142ms/step


2022-09-28 22:50:40.313766: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 22:50:40.488212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 152ms/step


2022-09-28 22:50:41.718736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-09-28 22:51:53.706013: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 22:51:53.840712: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 48ms/step


2022-09-28 22:51:53.974637: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-28 22:51:54.109846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 104ms/step


2022-09-28 22:51:54.270389: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 10/10 [01:14<00:00,  7.45s/it]


&nbsp;

## 정답 제출 파일생성

In [14]:
for k in tqdm(range(10)):
  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/{test_num}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:01<00:00,  5.23it/s]


- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [15]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.125000,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.280000,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,7.432846,7.611492,2.952911,-5.485766,-2.731483,-1.444200,-0.043641,0.000658,0.157395,15.553200,...,-9.942870,-0.000748,34.846863,-0.729149,52.851240,607.785000,0.015078,10.428878,0.634186,-25.900318
2,8.055674,12.937197,-2.894372,-9.631308,-6.345500,1.369160,-0.025701,0.002336,-0.053621,26.125135,...,-12.095488,-0.202013,40.507600,-1.660148,61.801200,873.518860,0.191536,14.218755,27.184510,-17.859830
3,13.133212,19.316584,34.227272,-18.201387,-10.604058,0.284268,0.011322,0.002296,-0.042184,26.543226,...,-12.308499,0.006875,47.348846,-3.666065,94.935330,956.699950,-0.416866,15.133969,50.766243,39.180943
4,7.427299,25.360119,59.181736,-20.365845,-13.249518,0.573949,0.006970,0.002254,-0.141424,35.624020,...,-10.450993,0.137144,61.657482,-6.364346,132.275000,1041.835400,-0.154059,14.582619,71.080980,66.116130
5,5.052385,31.959824,70.376330,-21.996851,-17.403135,3.174129,-0.030077,0.003456,0.076532,43.640170,...,-10.483763,-0.264305,69.237236,-6.551600,171.905730,1222.622700,-0.017677,15.905648,93.136620,92.366940
6,10.063150,36.245003,83.917650,-28.948133,-18.246305,6.135219,-0.060876,-0.001674,0.097426,44.943140,...,-10.732766,-0.059657,52.882850,-6.911216,233.928480,1137.916400,-0.083116,17.425910,113.575035,126.373410
7,15.265370,45.258590,71.035530,-24.236061,-23.261410,4.896489,-0.029947,-0.000698,0.128184,52.973362,...,-12.271767,0.104504,35.497370,-6.957687,300.241420,873.651700,0.296162,18.005106,134.117100,132.834700
8,12.436709,51.679996,73.192250,-20.153097,-29.053602,7.598347,-0.013254,0.001193,-0.257432,52.249930,...,-12.617627,-0.082060,42.614380,-7.862935,373.814240,1089.365400,0.005193,16.382843,151.769790,134.850430
9,13.191787,58.831497,51.074543,-28.562460,-30.789090,8.422217,-0.037427,0.000754,-0.286893,65.888940,...,-11.835744,-0.104684,55.385212,-7.547737,397.446960,1250.801000,0.080671,15.449335,153.794220,148.401670


- 변동률 계산 

In [16]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']

for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값
    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1])/base_d)
    
    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean())/base_d
    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

- sep 0  ~ sep 9 까지 합치기

In [17]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [18]:
answer_df_0

,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,...,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률,Set,일자
0,0.001920,0.005588,0.001015,-0.001613,-0.000692,-0.000156,-0.000016,1.957979e-07,0.000032,0.013259,...,0.031524,-0.000466,0.014546,0.111424,2.683371e-06,0.001994,0.000218,-0.012406,0,d+1
1,0.002081,0.009498,-0.000995,-0.002833,-0.001607,0.000148,-0.000009,6.949752e-07,-0.000011,0.022272,...,0.036645,-0.001060,0.017009,0.160140,3.408608e-05,0.002718,0.009358,-0.008555,0,d+2
2,0.003393,0.014181,0.011763,-0.005353,-0.002686,0.000031,0.000004,6.832559e-07,-0.000009,0.022628,...,0.042834,-0.002341,0.026128,0.175390,-7.418610e-05,0.002893,0.017475,0.018768,0,d+3
3,0.001919,0.018618,0.020339,-0.005990,-0.003356,0.000062,0.000003,6.705140e-07,-0.000029,0.030370,...,0.055778,-0.004063,0.036405,0.190997,-2.741655e-05,0.002788,0.024468,0.031670,0,d+4
4,0.001305,0.023463,0.024186,-0.006470,-0.004408,0.000343,-0.000011,1.028112e-06,0.000016,0.037203,...,0.062635,-0.004183,0.047312,0.224141,-3.145902e-06,0.003041,0.032060,0.044244,0,d+5
5,0.002600,0.026609,0.028840,-0.008514,-0.004622,0.000663,-0.000022,-4.979252e-07,0.000020,0.038314,...,0.047840,-0.004413,0.064382,0.208612,-1.479150e-05,0.003332,0.039095,0.060533,0,d+6
6,0.003943,0.033227,0.024413,-0.007128,-0.005892,0.000529,-0.000011,-2.077901e-07,0.000026,0.045160,...,0.032112,-0.004442,0.082632,0.160165,5.270546e-05,0.003442,0.046166,0.063628,0,d+7
7,0.003213,0.037941,0.025154,-0.005927,-0.007359,0.000821,-0.000005,3.549819e-07,-0.000052,0.044543,...,0.038551,-0.005020,0.102881,0.199711,9.240929e-07,0.003132,0.052243,0.064594,0,d+8
8,0.003408,0.043191,0.017553,-0.008401,-0.007799,0.000910,-0.000014,2.243340e-07,-0.000058,0.056170,...,0.050104,-0.004819,0.109385,0.229307,1.435638e-05,0.002954,0.052939,0.071085,0,d+9
9,0.003434,0.044676,0.020436,-0.011925,-0.009864,0.000895,-0.000018,-8.071937e-07,0.000015,0.070501,...,0.055545,-0.004349,0.125207,0.231692,1.868454e-05,0.002792,0.059447,0.079638,0,d+10


- 정답 양식으로 변경

In [19]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'./submit/submit_{test_num}.csv',index=False)

- 계산된 변동률 결과물

In [20]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.001920,0.005588,0.001015,-0.001613,-0.000692,-1.560632e-04,-1.606047e-05,1.957979e-07,...,-0.001151,-0.000001,0.031524,-0.000466,0.014546,0.111424,0.000003,0.001994,0.000218,-0.012406
1,0,d+2,0.002081,0.009498,-0.000995,-0.002833,-0.001607,1.479542e-04,-9.458385e-06,6.949752e-07,...,-0.001400,-0.000336,0.036645,-0.001060,0.017009,0.160140,0.000034,0.002718,0.009358,-0.008555
2,0,d+3,0.003393,0.014181,0.011763,-0.005353,-0.002686,3.071861e-05,4.166787e-06,6.832559e-07,...,-0.001424,0.000011,0.042834,-0.002341,0.026128,0.175390,-0.000074,0.002893,0.017475,0.018768
3,0,d+4,0.001919,0.018618,0.020339,-0.005990,-0.003356,6.202209e-05,2.564946e-06,6.705140e-07,...,-0.001209,0.000228,0.055778,-0.004063,0.036405,0.190997,-0.000027,0.002788,0.024468,0.031670
4,0,d+5,0.001305,0.023463,0.024186,-0.006470,-0.004408,3.430027e-04,-1.106886e-05,1.028112e-06,...,-0.001213,-0.000439,0.062635,-0.004183,0.047312,0.224141,-0.000003,0.003041,0.032060,0.044244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.002527,-0.013608,0.019204,0.011689,0.006708,7.288874e-07,-3.968332e-06,NaN,...,-0.000412,0.000600,0.009819,0.001135,0.015166,0.009088,0.000053,-0.007203,-0.000082,-0.044550
146,9,d+12,0.002873,-0.014256,0.020418,0.011905,0.006511,9.181387e-06,6.116244e-07,NaN,...,-0.000721,0.000970,0.009984,0.001195,0.020142,0.012414,0.000008,-0.009973,0.009759,-0.045528
147,9,d+13,0.003091,-0.013766,0.021909,0.012112,0.006173,5.108601e-06,-5.733798e-06,NaN,...,-0.000619,-0.001215,0.004867,0.001606,0.016800,0.011917,0.000069,-0.010579,0.013751,-0.045591
148,9,d+14,0.003473,-0.014012,0.021203,0.012680,0.007965,3.883336e-06,-6.315107e-06,NaN,...,-0.000722,0.001922,-0.004937,0.001520,0.017713,0.013050,-0.000017,-0.012831,0.015723,-0.045495


- 제출 양식

In [21]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [22]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.001920,0.005588,0.001015,-0.001613,-0.000692,-1.560632e-04,-1.606047e-05,1.957979e-07,...,-0.001151,-0.000001,0.031524,-0.000466,0.014546,0.111424,0.000003,0.001994,0.000218,-0.012406
1,0,d+2,0.002081,0.009498,-0.000995,-0.002833,-0.001607,1.479542e-04,NaN,6.949752e-07,...,-0.001400,-0.000336,0.036645,-0.001060,0.017009,0.160140,0.000034,0.002718,0.009358,-0.008555
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,4.166787e-06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.001919,0.018618,0.020339,-0.005990,-0.003356,6.202209e-05,NaN,6.705140e-07,...,-0.001209,0.000228,0.055778,-0.004063,0.036405,0.190997,-0.000027,0.002788,0.024468,0.031670
4,0,d+5,0.001305,0.023463,0.024186,-0.006470,-0.004408,3.430027e-04,NaN,1.028112e-06,...,-0.001213,-0.000439,0.062635,-0.004183,0.047312,0.224141,-0.000003,0.003041,0.032060,0.044244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.002527,-0.013608,0.019204,0.011689,0.006708,7.288874e-07,-3.968332e-06,NaN,...,-0.000412,0.000600,0.009819,0.001135,0.015166,0.009088,0.000053,-0.007203,-0.000082,-0.044550
146,9,d+12,0.002873,-0.014256,0.020418,0.011905,0.006511,9.181387e-06,6.116244e-07,NaN,...,-0.000721,0.000970,0.009984,0.001195,0.020142,0.012414,0.000008,-0.009973,0.009759,-0.045528
147,9,d+13,0.003091,-0.013766,0.021909,0.012112,0.006173,5.108601e-06,-5.733798e-06,NaN,...,-0.000619,-0.001215,0.004867,0.001606,0.016800,0.011917,0.000069,-0.010579,0.013751,-0.045591
148,9,d+14,0.003473,-0.014012,0.021203,0.012680,0.007965,3.883336e-06,-6.315107e-06,NaN,...,-0.000722,0.001922,-0.004937,0.001520,0.017713,0.013050,-0.000017,-0.012831,0.015723,-0.045495
